# Sales 

## Import packages

In [ ]:
# importing packages
import pandas as pd
import numpy as np
import utils

print("All packages imported successfully!")

## Import and inspect data

In [ ]:
# Dataframe from csv file
df = pd.read_csv("AusApparalSales4thQrt2020.csv")
print(f"df.head()\n {df.head()} \n")
print(f"df.tail() \n {df.tail()}")

In [ ]:
# df.describe()
print(f"size: {df.size}\n")
print(f"memory_usage: \n{df.memory_usage(deep=True)} \n")
print(f"{df.dtypes}")

In [ ]:
# Optimize memory
# Convert object type to str, Date to datetime and reduce int.
cols = df.select_dtypes(np.object_).columns[1:]  # every object type except Date
df[cols] = df[cols].astype("string")
df["Date"] = pd.to_datetime(df["Date"])
df["Unit"] = df["Unit"].astype("int8")
df["Sales"] = df["Sales"].astype("int32")

In [ ]:
# df.set_index(['Date'], inplace=True)
# df.reset_index(inplace=True)

In [ ]:
print("After optimizing")
print(f"size: {df.size}\n")
print(f"memory_usage: \n{df.memory_usage(deep=True)}\n")
print(f"{df.dtypes}")

**Observations**

We can see that at least in `Date`, 'Unit`and 'Sales` the memory_usage reduced. It wasn't the case for Time, State and Group. 

In [ ]:
# Changing Time, State, Group back to object
df[cols] = df[cols].astype(object)
print("After changing back")
print(f"memory_usage: \n{df.memory_usage(deep=True)}\n")
print(f"{df.dtypes}")

In [ ]:
dimensions = df.shape
print(
    f"Dimensions {dimensions} Data contains {dimensions[0]} rows and {dimensions[1]} columns"
)
print(f"Columns are {list(df.columns)}")

### Data wrangling

a. Ensure that the data is clean and free from any missing or incorrect entries.

In [ ]:
# Find rows with at least one NA value
df[df.isna().any(axis=1)]

# Find rows where all values are empty
# df[df.isna().all(axis=1)]

In [ ]:
# Find columns with at least one empty value
df.columns[df.isna().any()]

# Find columns where all values are empty
# df.columns[df.isna().all()]

In [ ]:
# Checking for null in any of the columns
# df.isna().sum()
df.notna().sum()

#### Observations

In the result, we can see that the data does not have null values:

- `df.isna().sum()`returns 0 for each column
- `df.notnat().sum()` returns 7560 for each columns, which tells me that there are not NaN

However, `isna()` or `df.notna()` doesn't seen to check for missing values (spaces, empty string)

So I added NaN and empty string to the dataframe to make sure I check for empty string as well as NaN values.  


In [ ]:
from datetime import datetime
import locale

# Note:
# I was getting "time data '29-Dec-2020' does not match format '%d-%b-%Y'""
# The %b directive in the strptime method is locale-dependent so I set it to EN-US

# Set the locale to English
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")

# 20-Dec-2020,'Afternoon', '', Seniors, 13, 32500
# 30-Dec-2020','Evening', 'TAS',None,12, 33500
# 31-Dec-2020',np.nan, 'TAS','Kids',12, 3350

# Add 3 rows with empty values, None and NaN to the dataframe
df2 = df.copy()
df2.loc[len(df2.index)] = [
    datetime.strptime("29-Dec-2020", "%d-%b-%Y"),
    "Afternoon",
    "",
    "Seniors",
    12,
    32500,
]  # empty value in State
df2.loc[len(df2.index)] = [
    datetime.strptime("30-Dec-2020", "%d-%b-%Y"),
    "Evening",
    "TAS",
    None,
    12,
    33500,
]  # None value in Group
df2.loc[len(df2.index)] = [
    datetime.strptime("31-Dec-2020", "%d-%b-%Y"),
    np.nan,
    "TAS",
    "Kids",
    12,
    33500,
]  # Nan value in Time
df2.tail()

In [ ]:
print(f"Before optimization")
print(f"df2.dtypes: \n{df2.dtypes}\n")
print(f"memory_usage: \n{df2.memory_usage(deep=True)}\n")

# Optimize df2 too 
df2["Unit"] = df2["Unit"].astype("int8")
df2["Sales"] = df2["Sales"].astype("int32")

print(f"After optimization")
print(f"df2.dtypes: \n{df2.dtypes}\n")
print(f"memory_usage: \n{df2.memory_usage(deep=True)}\n")

# df2.select_dtypes(np.object_).columns

In [ ]:
# Checking for null with numpy
print(
    f"Using numPy isnull(), it only detects None and NaN \nSo running 'np.where(pd.isnull(df2)' returns: {np.where(pd.isnull(df2))}"
)
print(
    "Indicating that in row 7561 column 3 there is a None value,  and in row 7562 column 3 there is a NaN value  \n"
)

print(
    f"Same as isna(), it only detects None and NaN\n So running, 'df2[df2.isna().any(axis=1)])': \n{df2[df2.isna().any(axis=1)]} \n"
)

print(
    f"In conclusion, we need to check for '' (empty) values as well df2[df2.map(lambda x: x == '').any(axis=1)]\n"
)
print(f"{df2[df2.map(lambda x: x == '').any(axis=1)]} \n")
print(f"And we combine checking for NaN and '' empty values to ")

# Using isna()
# df2[df2.isna().any(axis=1) | df2.map(lambda x: x == '').any(axis=1)]

# Using isnull
df2[df2.isnull().any(axis=1) | (df2 == "").any(axis=1)]

In [ ]:
# To check for the columns
# cols_with_empty = df2.apply(lambda col: col.apply(lambda x: x.strip() == '' if isinstance(x, str) else False).any())
# print(cols_with_empty[cols_with_empty].index.tolist())
df2.columns[df2.isnull().any() | (df2 == "").any()]

#### Observations

Using numpy we can see:

- `np.where(pd.isnull(df))` returns the row and column indices where the value is NaN. We get empty arrays as a result
- `np.where(df.map(lambda x: x == ''))` also returns empty arrays. 

Note that using `map` requires calling a Python function once for each cell of the DataFrame. That could be slow for a large DataFrame, so it would be better 
to arrange for all the blank cells to contain NaN instead, and then use `pd.isnull()`


b. Based on your knowledge of Data Analytics, include your recommendations for treating missing and incorrect data (dropping the null values or filling them).

#### Recomendations

There are a couple things we could do. 
- Per above comment, for all the blank cells I will fill with NaN 

In [ ]:
# Replace with NaN and then find those rows.
df = df.replace(" ", np.nan)
nan_values = df[df.isna().any(axis=1)]
nan_values

nan_rows = df.loc[df.isna().any(axis=1)]
nan_rows

In [ ]:
# I will use df2 to make my recommendation in the case of missing/incorrect data

# 1. First replace '' with NaN

# df2.replace('None', np.nan, inplace=True) and find them
df2 = df2.replace("", np.nan)

# Find Options
# a. Using `df.loc``
# nan_rows  = df2.loc[df2.isna().any(axis=1)]

# b. finding all `isnull()` rows first
# null_mask = df2.isnull().any(axis=1)
# null_rows = df2[null_mask]
# null_rows

# c. or directly finding all `isna()`` rows
nan_rows = df2[df2.isna().any(axis=1)]
nan_rows

In [ ]:
# 2. Then, make sure that each value is unique, so we can groupby later.
categorical = ["Time", "State", "Group"]
df2_categorical = df2[categorical]

# a. describe() will give you the unique info
df2_categorical.describe()

In [ ]:
df2_categorical.info()
df2.dtypes

In [ ]:
# b. Or inspecting with `nunique()` and `unique()`
if df2.Time.nunique() != len(df2.Time.unique()):
    print(f"Time values {df2.Time.unique()} - {df2.Time.nunique()}")

if df2.State.nunique() != len(df2.State.unique()):
    print(f"State values {df2.State.unique()} - {df2.State.nunique()}")

if df2.Group.nunique() != len(df2.Group.unique()):
    print(f"Group values {df2.Group.unique()} - {df2.Group.nunique()}")

#### Observations
We can see that some categorical values has spaces. For example, these are two different values.
- For Time, ' Evening' and 'Evening'  
- For State, ' TAS' isn't the same as 'TAS'
- For Group, ' Kids'and 'Kids'. ' Seniors' vs 'Seniors'

In [ ]:
# c. with value_counts() we can also see that something isn't quite right. (What are the possible values that belogn to this category)
for colName in list(df2_categorical):
    print(f"{colName} counts: \n {df2[colName].value_counts()} \n")

In [ ]:
times = df2.loc[7560: 7564]['Time']
states = df2.loc[7560: 7564]['State']
group = df2.loc[7560: 7564]['Group']

print(times)
print(states)
print(group)

In [ ]:
# 3. Remove spaces to then groupby unique values
# ====================
# Not sure why this isn't working
# for colName in list(df2_categorical):
#     df[colName] = df[colName].apply(lambda x: x.strip())
#     print(f"{colName} values: {df2[colName].unique()} - {df2[colName].nunique()}")

df2["Time"] = df2["Time"].str.strip()
df2["State"] = df2["State"].str.strip()
df2["Group"] = df2["Group"].str.strip()

print(f"Time values {df2.Time.unique()} - {df2.Time.nunique()}")
print(f"State values {df2.State.unique()} - {df2.State.nunique()}")
print(f"Group values {df2.Group.unique()} - {df2.Group.nunique()}")
print("\nDescribe df2 after removing spaces")
df2[categorical].describe() 


In [ ]:
# Apply the same to the original df
df["Time"] = df["Time"].str.strip()
df["State"] = df["State"].str.strip()
df["Group"] = df["Group"].str.strip()
print("\nDescribe df after removing spaces")
df[categorical].describe() 

In [ ]:
# 4. In this case I decided to dropna() but you could also fillin with mode() sample because we only have 3 rows (added) with NaN

# time_value =list(df2['Time'].mode().sample())[0]
# df2.fillna({'Time': time_value }, inplace=True)

# group_value =list(df2['Group'].mode().sample())[0]
# df2.fillna({'Group': group_value }, inplace=True)

# state_value =list(df2['State'].mode().sample())[0]
# df2.fillna({'State': group_value }, inplace=True)
df2 = df2.dropna
df = df.dropna()

c. Choose a suitable Data Wrangling technique——either data standardization or normalization.
#### Observations
I will use groupby() since we only have one dataset to split my data into categories

In [ ]:
# Group by categoricals
cat_time = df.groupby("Time")
cat_group = df.groupby("Group")
cat_state = df.groupby("State")

In [ ]:
# Group by month & year
if 'Date'in df.index.names:
    df.reset_index(inplace=True)

# Make sure the 'Date' column is in datetime format
df['Date'] = pd.to_datetime(df['Date'])
df['YearMonth'] = df['Date'].dt.to_period('M')

cat_month_year = df.groupby('YearMonth')
sales_by_month_year = cat_month_year['Sales'].sum().reset_index()
sales_by_month_year

#### Observations
- Here we can see that we only have three months. 
- We observe that Dec has more sales compared to Oct and Nov.

### Data analysis

a. Perform descriptive statistical analysis on the data in the Sales and Unit columns. Utilize techniques such as mean, median, mode, and standard deviation for this analysis.

In [ ]:
print(f"Sales.describe():\n{df.Sales.describe()}\n")
print(f"Total Sales: {df['Sales'].sum()}\n")
print(f"Total Sales per month: \n{cat_month_year['Sales'].sum().reset_index()}\n")
# Based on each category
print(f"By Group:\n {cat_group.Sales.agg(['mean', 'sum'])}\n")
print(f"By Time:\n {cat_time.Sales.agg(['mean', 'sum'])}\n")
print(f"By State:\n {cat_state.Sales.agg(['mean', 'sum'])}\n")

# print(f"Sales median: {df['Sales'].median()}")
# # print(f"Sales mean: {df['Sales'].mean()}")
# print(f"Sales mode: {df['Sales'].mode()} \n")

print(f"Unit.describe():\n{df.Unit.describe()}\n")
print(f"Total unit: {df['Unit'].sum()}\n")
print(f"Total units per month: \n{cat_month_year['Unit'].sum().reset_index()}\n")
# Based on each category
print(f"By Group:\n {cat_group.Unit.agg(['mean', 'sum'])}\n")
print(f"By Time:\n {cat_time.Unit.agg(['mean', 'sum'])}\n")
print(f"By State:\n {cat_state.Unit.agg(['mean', 'sum'])}\n")

# print(f"Unit median: {df['Unit'].median()}")
# print(f"Unit mean: {df['Unit'].mean()}")
# print(f"Unit mode: {df['Unit'].mode()}")

b. Identify the group with the highest sales and the group with the lowest sales based on the data provided.

c. Identify the group within the highest and lowest sales based on the data provided.


In [ ]:
cat_group.Sales.sum().nlargest(5)

#### Observations
- The group with highest sales is Men.
- The group with the lowest sales is Seniours.
- The groups within the highest and lowest sales are Women and Kids.

d. Generate weekly, monthly, and quarterly reports to document and present the results of the analysis conducted.

In [123]:
# Create a new columns 'YearQuarter' and 'YearWeek' to hold the year, quarter and week number for each date

df['YearQuarter'] = df['Date'].dt.to_period('Q')
df['YearWeek'] = df['Date'].dt.strftime('%Y-%U')

# Describe each group
print(f"Weekly: {df.groupby('YearWeek').Sales.describe()} \n")
print(f"Monthly: {df.groupby('YearMonth').Sales.describe()}\n")
print(f"Quarterly: {df.groupby('YearQuarter').Sales.describe()}")

Weekly:           count          mean           std      min      25%      50%  \
YearWeek                                                                 
2020-39   252.0  44940.476190  30401.470867   7500.0  21875.0  33750.0   
2020-40   588.0  45663.265306  29662.463059   7500.0  22500.0  37500.0   
2020-41   588.0  45680.272109  29957.817907   7500.0  22500.0  35000.0   
2020-42   588.0  45225.340136  29520.627513   7500.0  22500.0  35000.0   
2020-43   504.0  44965.277778  30211.502370   7500.0  20000.0  33750.0   
2020-44   588.0  35161.564626  26608.306249   5000.0  15000.0  25000.0   
2020-45   588.0  36156.462585  27559.401709   5000.0  15000.0  25000.0   
2020-46   588.0  36339.285714  27706.605418   5000.0  15000.0  25000.0   
2020-47   588.0  35901.360544  27185.080176   5000.0  15000.0  25000.0   
2020-48   588.0  48975.340136  34674.229649   5000.0  22500.0  37500.0   
2020-49   588.0  52687.074830  35826.917392  12500.0  25000.0  37500.0   
2020-50   588.0  54094.387755 